In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import warnings
warnings.filterwarnings('ignore') 

from tensorflow import keras
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score
from Models import SAE, CNN_AE, LSTM_AE, GRU_AE, Bi_LSTM_AE, CNN_Bi_LSTM_AE, Causal_CNN_AE, Wavenet, Attention_Bi_LSTM_AE, Attention_CNN_Bi_LSTM_AE, Attention_Wavenet

np.random.seed(7)
tf.random.set_seed(7)

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [3]:
svm_reg = SVR(gamma='scale')

In [4]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
dataset_path = './datasets/regression/'

In [5]:
def n_steps_reshape(X_train_full, y_train_full, n_steps=10, for_rnn=False):
    new_data = []
    new_label = []
    columns = X_train_full.columns
    for i in range(X_train_full.shape[0]-n_steps):
        new_instance = []
        train_data = X_train_full[i:i+n_steps]
        for c in columns:
            for v in train_data[c].values:
                new_instance.append(v)
#         for _, row in train_data.iterrows():
#             for c in columns:
#                 new_instance.append(row[c])
        new_label.append(y_train_full[i+n_steps])
        new_data.append(new_instance)

    scaler = RobustScaler()
    new_data = scaler.fit_transform(new_data)
    new_label = scaler.fit_transform(np.array(new_label).reshape(-1,1))

    if for_rnn:
        return np.array(new_data).reshape(len(new_data), n_steps, columns.shape[0]), new_label
    else:
        return np.array(new_data), new_label

In [6]:
def LSTM_Model(n_steps, n_features):
    return keras.models.Sequential([
        keras.layers.LSTM(128, return_sequences=True, input_shape=[n_steps, n_features]),
        keras.layers.LSTM(128),
        keras.layers.Dense(1, activation=keras.layers.LeakyReLU(alpha=0.5))
    ])

In [7]:
results = []

In [36]:
from TRepNet import TRepNet

def get_codings(X_train, n_steps, n_features):
#     X_train, X_test, n_steps = flatten_ts(train_x, test_x)
#     X_train, X_test = rnn_reshape(X_train, X_test, n_steps // n_features, n_features)
    encoder, decoder = TRepNet(n_steps, n_features, activation='elu')
    model = keras.models.Sequential([encoder, decoder])
    
    model.compile(loss="mae", optimizer=keras.optimizers.Nadam(lr=0.001, clipnorm=1.), metrics=['mae'])
    history = model.fit(X_train, X_train, epochs=500, batch_size=16, validation_split=0.20, callbacks=[es], verbose=1, shuffle=False)

    # Codings
    return encoder.predict(X_train)

In [9]:
es = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [10]:
# for fn in [SAE, CNN_AE, LSTM_AE, GRU_AE, Bi_LSTM_AE, CNN_Bi_LSTM_AE, Causal_CNN_AE, Wavenet, TRepNet]:
#     results = []

#     print(fn.__name__)
    
#     name = 'Solar Generation'
#     solar_data = pd.read_csv(dataset_path + 'Solar/data.csv', quotechar='"').fillna(0)
#     solar_data_X = solar_data.drop(columns=['SITE_NO', 'DATE', 'TIME'])
#     solar_data_y = solar_data['GEN_ENERGY']
#     X_train_full, y_train_full = n_steps_reshape(solar_data_X, solar_data_y, 10, for_rnn=True)
#     X_train_full = get_codings(X_train_full, 10, X_train_full.shape[2])
#     X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
#     svm_reg.fit(X_train, y_train)
#     pred = svm_reg.predict(X_test)
#     print(mse(y_test, pred))
#     results.append({'dataset': name, 'MSE': mse(y_test, pred)})
    
#     name = 'Beijing PM 2.5'
#     beijing_data = pd.read_csv(dataset_path + 'Beijing-PM25.csv').dropna().drop(columns=['No', 'year']).reset_index(drop=True)
#     beijing_data_X = pd.get_dummies(beijing_data, columns=['cbwd'])
#     beijing_data_y = beijing_data['pm2.5']
#     X_train_full, y_train_full = n_steps_reshape(beijing_data_X, beijing_data_y, 10, for_rnn=True)
#     X_train_full = get_codings(X_train_full, 10, X_train_full.shape[2])
#     X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
#     svm_reg.fit(X_train, y_train)
#     pred = svm_reg.predict(X_test)
#     print(mse(y_test, pred))
#     results.append({'dataset': name, 'MSE': mse(y_test, pred)})
    
#     name = 'Appliance Energy Prediction'
#     energy_data = pd.read_csv(dataset_path + 'energydata_complete.csv')
#     enery_data_X = energy_data.drop(columns=['date'])
#     enery_data_y = energy_data['Appliances']
#     X_train_full, y_train_full = n_steps_reshape(enery_data_X, enery_data_y, 10, for_rnn=True)
#     X_train_full = get_codings(X_train_full, 10, X_train_full.shape[2])
#     X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
#     svm_reg.fit(X_train, y_train)
#     pred = svm_reg.predict(X_test)
#     print(mse(y_test, pred))
#     results.append({'dataset': name, 'MSE': mse(y_test, pred)})
    
#     name = 'Parking Birmingham'
#     parking_data = pd.read_csv(dataset_path + 'Parking Birmingham.csv')
#     parking_data_X = parking_data.drop(columns=['SystemCodeNumber', 'LastUpdated'])
#     parking_data_y = parking_data['Occupancy']
#     X_train_full, y_train_full = n_steps_reshape(parking_data_X, parking_data_y, 10, for_rnn=True)
#     X_train_full = get_codings(X_train_full, 10, X_train_full.shape[2])
#     X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
#     svm_reg.fit(X_train, y_train)
#     pred = svm_reg.predict(X_test)
#     print(mse(y_test, pred))
#     results.append({'dataset': name, 'MSE': mse(y_test, pred)})
    
#     name = 'Daily Deemand Forecasting'
#     demand_data = pd.read_csv(dataset_path + 'Daily_Demand_Forecasting_Orders.csv', sep=';')
#     demand_data_X = demand_data
#     demand_data_y = demand_data['Target']
#     X_train_full, y_train_full = n_steps_reshape(demand_data_X, demand_data_y, 10, for_rnn=True)
#     X_train_full = get_codings(X_train_full, 10, X_train_full.shape[2])
#     X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
#     svm_reg.fit(X_train, y_train)
#     pred = svm_reg.predict(X_test)
#     print(mse(y_test, pred))
#     results.append({'dataset': name, 'MSE': mse(y_test, pred)})
    
#     pd.DataFrame(results).to_csv('./results/regression-'+ fn.__name__ +'-results.csv', index=False)
# print('END')

### Solar Generation

In [11]:
solar_data = pd.read_csv(dataset_path + 'Solar/data.csv', quotechar='"').fillna(0)
solar_data_X = solar_data.drop(columns=['SITE_NO', 'DATE', 'TIME'])
solar_data_y = solar_data['GEN_ENERGY']
X_train_full, y_train_full = n_steps_reshape(solar_data_X, solar_data_y, 10)
X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
svm_reg.fit(X_train, y_train)
pred = svm_reg.predict(X_test)
print(mse(y_test, pred))

0.021132202063766075


In [12]:
X_train, X_test = X_train.reshape(X_train.shape[0], 10, solar_data_X.shape[1]), X_test.reshape(X_test.shape[0], 10, solar_data_X.shape[1])

In [13]:
model = LSTM_Model(X_train.shape[1], X_train.shape[2])
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, shuffle=False, callbacks=[es])

Train on 39514 samples, validate on 4391 samples
Epoch 1/10
39514/39514 [==============================] - 23s 588us/sample - loss: 0.1823 - mse: 0.1823 - val_loss: 0.0337 - val_mse: 0.0337
Epoch 2/10
39514/39514 [==============================] - 14s 350us/sample - loss: 0.1681 - mse: 0.1681 - val_loss: 0.0265 - val_mse: 0.0265
Epoch 3/10
39514/39514 [==============================] - 13s 323us/sample - loss: 0.1595 - mse: 0.1595 - val_loss: 0.0237 - val_mse: 0.0237
Epoch 4/10
39514/39514 [==============================] - 13s 331us/sample - loss: 0.1549 - mse: 0.1549 - val_loss: 0.0190 - val_mse: 0.0190
Epoch 5/10
39514/39514 [==============================] - 12s 306us/sample - loss: 0.1535 - mse: 0.1535 - val_loss: 0.0167 - val_mse: 0.0167
Epoch 6/10
39514/39514 [==============================] - 13s 319us/sample - loss: 0.1538 - mse: 0.1538 - val_loss: 0.0166 - val_mse: 0.0166
Epoch 7/10
39514/39514 [==============================] - 12s 297us/sample - loss: 0.1511 - mse: 0.1511 -

In [14]:
pred = model.predict(X_test)
mse(y_test, pred)

0.021210585206084046

### Beijing PM 2.5

In [15]:
beijing_data = pd.read_csv(dataset_path + 'Beijing-PM25.csv').dropna().drop(columns=['No', 'year']).reset_index(drop=True)
beijing_data_X = pd.get_dummies(beijing_data, columns=['cbwd'])
beijing_data_y = beijing_data['pm2.5']
X_train_full, y_train_full = n_steps_reshape(beijing_data_X, beijing_data_y, 10)

X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
svm_reg.fit(X_train, y_train)
pred = svm_reg.predict(X_test)
print(mse(y_test, pred))

0.04616497588589663


In [16]:
X_train, X_test, y_train, y_test = X_train.reshape(X_train.shape[0], 10, beijing_data_X.shape[1]), X_test.reshape(X_test.shape[0], 10, beijing_data_X.shape[1]), np.array(y_train), np.array(y_test)

In [17]:
model = LSTM_Model(X_train.shape[1], X_train.shape[2])
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, shuffle=False, callbacks=[es])

Train on 30057 samples, validate on 3340 samples
Epoch 1/10
30057/30057 [==============================] - 14s 481us/sample - loss: 0.1118 - mse: 0.1118 - val_loss: 0.0915 - val_mse: 0.0915
Epoch 2/10
30057/30057 [==============================] - 8s 264us/sample - loss: 0.0689 - mse: 0.0689 - val_loss: 0.0678 - val_mse: 0.0678
Epoch 3/10
30057/30057 [==============================] - 8s 259us/sample - loss: 0.0634 - mse: 0.0634 - val_loss: 0.0560 - val_mse: 0.0560
Epoch 4/10
30057/30057 [==============================] - 8s 263us/sample - loss: 0.0592 - mse: 0.0592 - val_loss: 0.0508 - val_mse: 0.0508
Epoch 5/10
30057/30057 [==============================] - 8s 257us/sample - loss: 0.0565 - mse: 0.0565 - val_loss: 0.0491 - val_mse: 0.0491
Epoch 6/10
30057/30057 [==============================] - 8s 259us/sample - loss: 0.0545 - mse: 0.0545 - val_loss: 0.0482 - val_mse: 0.0482
Epoch 7/10
30057/30057 [==============================] - 8s 263us/sample - loss: 0.0530 - mse: 0.0530 - val_l

In [18]:
pred = model.predict(X_test)
mse(y_test, pred)

0.052009185489119016

### Energy Data

In [38]:
energy_data = pd.read_csv(dataset_path + 'energydata_complete.csv')
enery_data_X = energy_data.drop(columns=['date'])
enery_data_y = energy_data['Appliances']
X_train_full, y_train_full = n_steps_reshape(enery_data_X, enery_data_y, 10, for_rnn=True)
X_train_full = get_codings(X_train_full, 10, X_train_full.shape[2])
X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
svm_reg.fit(X_train, y_train)
pred = svm_reg.predict(X_test)
print(mse(y_test, pred))

Train on 15780 samples, validate on 3945 samples
Epoch 1/500
15780/15780 [==============================] - 258s 16ms/sample - loss: 0.4215 - mae: 0.4215 - val_loss: 0.8134 - val_mae: 0.8134
Epoch 2/500
15780/15780 [==============================] - 198s 13ms/sample - loss: 0.3698 - mae: 0.3698 - val_loss: 0.8269 - val_mae: 0.8269
Epoch 3/500
15780/15780 [==============================] - 198s 13ms/sample - loss: 0.3537 - mae: 0.3537 - val_loss: 0.7886 - val_mae: 0.7886
Epoch 4/500
15780/15780 [==============================] - 208s 13ms/sample - loss: 0.3432 - mae: 0.3432 - val_loss: 0.7836 - val_mae: 0.7836
Epoch 5/500
15780/15780 [==============================] - 212s 13ms/sample - loss: 0.3356 - mae: 0.3356 - val_loss: 0.8037 - val_mae: 0.8037
Epoch 6/500
15780/15780 [==============================] - 198s 13ms/sample - loss: 0.3319 - mae: 0.3319 - val_loss: 0.7722 - val_mae: 0.7722
Epoch 7/500
15780/15780 [==============================] - 196s 12ms/sample - loss: 0.3266 - mae: 0

In [20]:
X_train, X_test, y_train, y_test = X_train.reshape(X_train.shape[0], 10, enery_data_X.shape[1]), X_test.reshape(X_test.shape[0], 10, enery_data_X.shape[1]), np.array(y_train), np.array(y_test)

In [21]:
model = LSTM_Model(X_train.shape[1], X_train.shape[2])
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, shuffle=False, callbacks=[es])

Train on 14202 samples, validate on 1578 samples
Epoch 1/10
14202/14202 [==============================] - 11s 754us/sample - loss: 2.3596 - mse: 2.3596 - val_loss: 2.2066 - val_mse: 2.2066
Epoch 2/10
14202/14202 [==============================] - 4s 302us/sample - loss: 1.9699 - mse: 1.9699 - val_loss: 2.1274 - val_mse: 2.1274
Epoch 3/10
14202/14202 [==============================] - 4s 285us/sample - loss: 1.8586 - mse: 1.8586 - val_loss: 2.1824 - val_mse: 2.1824
Epoch 4/10
14202/14202 [==============================] - 4s 284us/sample - loss: 1.7828 - mse: 1.7828 - val_loss: 2.1673 - val_mse: 2.1673
Epoch 5/10
14202/14202 [==============================] - 4s 291us/sample - loss: 1.7056 - mse: 1.7056 - val_loss: 2.0879 - val_mse: 2.0879
Epoch 6/10
14202/14202 [==============================] - 4s 280us/sample - loss: 1.6557 - mse: 1.6557 - val_loss: 2.0062 - val_mse: 2.0062
Epoch 7/10
14202/14202 [==============================] - 4s 288us/sample - loss: 1.6291 - mse: 1.6291 - val_l

In [22]:
pred = model.predict(X_test)
mse(y_test, pred)

1.980028086438499

### Parking Birmingham

In [23]:
parking_data = pd.read_csv(dataset_path + 'Parking Birmingham.csv')
parking_data_X = parking_data.drop(columns=['SystemCodeNumber', 'LastUpdated'])
parking_data_y = parking_data['Occupancy']
X_train_full, y_train_full = n_steps_reshape(parking_data_X, parking_data_y, 10)

X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
svm_reg.fit(X_train, y_train)
pred = svm_reg.predict(X_test)
print(mse(y_test, pred))

0.0347304968433626


In [24]:
X_train, X_test, y_train, y_test = X_train.reshape(X_train.shape[0], 10, parking_data_X.shape[1]), X_test.reshape(X_test.shape[0], 10, parking_data_X.shape[1]), np.array(y_train), np.array(y_test)

In [25]:
model = LSTM_Model(X_train.shape[1], X_train.shape[2])
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, shuffle=False, callbacks=[es])

Train on 25708 samples, validate on 2857 samples
Epoch 1/10
25708/25708 [==============================] - 14s 531us/sample - loss: 0.1127 - mse: 0.1127 - val_loss: 0.0607 - val_mse: 0.0607
Epoch 2/10
25708/25708 [==============================] - 7s 285us/sample - loss: 0.0532 - mse: 0.0532 - val_loss: 0.0479 - val_mse: 0.0479
Epoch 3/10
25708/25708 [==============================] - 7s 286us/sample - loss: 0.0488 - mse: 0.0488 - val_loss: 0.0447 - val_mse: 0.0447
Epoch 4/10
25708/25708 [==============================] - 7s 282us/sample - loss: 0.0454 - mse: 0.0454 - val_loss: 0.0424 - val_mse: 0.0424
Epoch 5/10
25708/25708 [==============================] - 7s 274us/sample - loss: 0.0428 - mse: 0.0428 - val_loss: 0.0406 - val_mse: 0.0406
Epoch 6/10
25708/25708 [==============================] - 7s 279us/sample - loss: 0.0401 - mse: 0.0401 - val_loss: 0.0389 - val_mse: 0.0389
Epoch 7/10
25708/25708 [==============================] - 7s 276us/sample - loss: 0.0376 - mse: 0.0376 - val_l

In [26]:
pred = model.predict(X_test)
mse(y_test, pred)

0.027566600022463545

### Daily Demand

In [27]:
demand_data = pd.read_csv(dataset_path + 'Daily_Demand_Forecasting_Orders.csv', sep=';')
demand_data_X = demand_data
demand_data_y = demand_data['Target']
X_train_full, y_train_full = n_steps_reshape(demand_data_X, demand_data_y, 10)

X_train, X_test, y_train, y_test = train_test_split(X_train_full, y_train_full, test_size=0.20, random_state=7)
svm_reg.fit(X_train, y_train)
pred = svm_reg.predict(X_test)
print(mse(y_test, pred))

1.4818427980593791


In [28]:
X_train, X_test, y_train, y_test = X_train.reshape(X_train.shape[0], 10, demand_data_X.shape[1]), X_test.reshape(X_test.shape[0], 10, demand_data_X.shape[1]), np.array(y_train), np.array(y_test)

In [29]:
model = LSTM_Model(X_train.shape[1], X_train.shape[2])
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1, shuffle=False, callbacks=[es])

Train on 36 samples, validate on 4 samples
Epoch 1/10
36/36 [==============================] - 7s 202ms/sample - loss: 0.6588 - mse: 0.6588 - val_loss: 0.2509 - val_mse: 0.2509
Epoch 2/10
36/36 [==============================] - 0s 2ms/sample - loss: 0.5866 - mse: 0.5866 - val_loss: 0.2805 - val_mse: 0.2805
Epoch 3/10
36/36 [==============================] - 0s 2ms/sample - loss: 0.5192 - mse: 0.5192 - val_loss: 0.3176 - val_mse: 0.3176
Epoch 4/10
36/36 [==============================] - 0s 2ms/sample - loss: 0.4640 - mse: 0.4640 - val_loss: 0.3429 - val_mse: 0.3429
Epoch 5/10
36/36 [==============================] - 0s 2ms/sample - loss: 0.4172 - mse: 0.4172 - val_loss: 0.3804 - val_mse: 0.3804
Epoch 6/10
36/36 [==============================] - 0s 2ms/sample - loss: 0.3776 - mse: 0.3776 - val_loss: 0.4345 - val_mse: 0.4345


In [30]:
pred = model.predict(X_test)
mse(y_test, pred)

1.7599617685253022